In [1]:
import torch

# Тесты моделей

In [2]:
rnn = torch.nn.LSTM(50, 20, 4)
input_tensor = torch.randn(5, 3, 50)
h0 = torch.randn(4, 3, 20)
c0 = torch.randn(4, 3, 20)
output, (hn, cn) = rnn(input_tensor, (h0, c0))
output.shape, hn.shape, cn.shape

(torch.Size([5, 3, 20]), torch.Size([4, 3, 20]), torch.Size([4, 3, 20]))

In [3]:
input_tensor = torch.randint(0, 10, (2, 5)).long()+4
padded = torch.nn.functional.pad(input=input_tensor, pad=(0, 5), mode='constant', value=2)
emb = torch.nn.Embedding(
    num_embeddings=14,
    embedding_dim=20,
    padding_idx=2
)
embeded = emb(input_tensor)
print(emb(input_tensor).shape)



torch.Size([2, 5, 20])


In [4]:
input_tensor

tensor([[13,  4,  5,  8,  5],
        [12, 11, 12,  6,  7]])

# Чистый Encoder

In [28]:
class DecoderLSTM(torch.nn.Module):
    def __init__(
        self, 
        embedding_size: int,  # размерность векторов в который преобразуется какждый элемент последовательности 
        hidden_size: int,  # размерность векторов на выходе
        num_layers: int,  # число скрытых состояний
        output_size: int,  # размер выхода
    ):
        super(DecoderLSTM, self).__init__()
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.LSTM = torch.nn.LSTM(
            self.embedding_size, 
            hidden_size, num_layers
        )
        self.fc = torch.nn.Linear(
            self.hidden_size, 
            self.output_size
        )

    def forward(
        self, 
        x,  # токен последоательности
        hidden_state,  # предыдущие скрытые состояния
        cell_state  # предыдущие состояния ячейки (короткой памяти)
    ):
        x = x.unsqueeze(0)  # x.shape == (1, 1, 31)
        outputs, (hidden_state, cell_state) = self.LSTM(x, (hidden_state, cell_state))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden_state, cell_state

decoder_embedding_size = 31
hidden_size = 17
num_layers = 7
output_size = 41

decoder_lstm = DecoderLSTM(decoder_embedding_size, hidden_size, num_layers, output_size)
print(decoder_lstm)

DecoderLSTM(
  (LSTM): LSTM(31, 17, num_layers=7)
  (fc): Linear(in_features=17, out_features=41, bias=True)
)


In [29]:
input_tensor = torch.randn(1, 31)
hidden_state = torch.randn(7, 1, 17)
cell_state = torch.randn(7, 1, 17)
outputs, hidden_state, cell_state = decoder_lstm(input_tensor, hidden_state, cell_state)
padded.shape, outputs.shape, hidden_state.shape, cell_state.shape 

(torch.Size([2, 10]),
 torch.Size([1, 41]),
 torch.Size([7, 1, 17]),
 torch.Size([7, 1, 17]))

# Со встроенной векторизацией

In [21]:
class DecoderLSTM(torch.nn.Module):
    def __init__(
        self, 
        vocab_size: int,  # размер словаря  
        embedding_size: int,  # размерность векторов в который преобразуется какждый элемент последовательности 
        hidden_size: int,  # размерность векторов на выходе
        num_layers: int,  # число скрытых состояний
        output_size: int,  # размер выхода
        pad_idx: int=2
    ):
        super(DecoderLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.embedding = torch.nn.Embedding(
            self.vocab_size, 
            self.embedding_size,
            padding_idx=pad_idx
        )
        self.LSTM = torch.nn.LSTM(
            self.embedding_size, 
            hidden_size, num_layers
        )
        self.fc = torch.nn.Linear(
            self.hidden_size, 
            self.output_size
        )

    def forward(
        self, 
        x,  # токен последоательности
        hidden_state,  # предыдущие скрытые состояния
        cell_state  # предыдущие состояния ячейки (короткой памяти)
    ):
        x = x.unsqueeze(0)  # x.shape == (1, 1)
        embedding = self.embedding(x)  # embedding.shape == (1, 1, 31)
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden_state, cell_state

vocab_size_decoder = 23
decoder_embedding_size = 31
hidden_size = 17
num_layers = 7
output_size = 41

decoder_lstm = DecoderLSTM(vocab_size_decoder, decoder_embedding_size, hidden_size, num_layers, output_size)
print(decoder_lstm)

DecoderLSTM(
  (embedding): Embedding(23, 31, padding_idx=2)
  (LSTM): LSTM(31, 17, num_layers=7)
  (fc): Linear(in_features=17, out_features=41, bias=True)
)


In [24]:
input_tensor = torch.randint(4, 23, (1, 1)).long()[0]
hidden_state = torch.randn(7, 1, 17)
cell_state = torch.randn(7, 1, 17)
outputs, hidden_state, cell_state = decoder_lstm(input_tensor, hidden_state, cell_state)
padded.shape, outputs.shape, hidden_state.shape, cell_state.shape 

(torch.Size([2, 10]),
 torch.Size([1, 41]),
 torch.Size([7, 1, 17]),
 torch.Size([7, 1, 17]))